# Pushup Circle
## Part 1: The Pushup Circle
### Section 1: Problem statement
Suppose $n$ friends are in a circle. At time $0$ one friend does one pushup. At time $t \in \mathbb{N}$ each friend does one pushup if exactly one of their immediate neighbors did a pushup at time $t - 1$. Otherwise, they do not do any pushups. Let $C_n(t)$ be the number of pushups done at time $t$, and let $P(n) = \sum_{t=0}^\infty C_n(t)$ be the total number of pushups (possibly infinity) done with a group of $n$ friends.

For what values of $n$ is $P(n) < \infty$?

### Section 2: Some useful functions

In [1]:
# rotate the bits of an integer, N is the bit length of an integer
def bit_rotate_right(N, number, shift):
    return (2 ** N - 1) & (number >> shift | number << (N-shift))

def bit_rotate_left(N, number, shift):
    return bit_rotate_right(N, number, N - shift)

def count_ones(N, number):
    return number.bit_count()

def xor(N, a, b):
    return a ^ b

def print_bits(N, number, as_blocks=False):
    string = bin(number)[2:].zfill(N)
    if as_blocks:
        print(string.replace('0', ' ').replace('1', '■'))
    else:
        print(string)

N = 7
num = 0b1011010
print_bits(N, num)
print_bits(N, bit_rotate_right(N, num, 1))
print_bits(N, xor(N, num, bit_rotate_right(N, num, 1)))
print_bits(N, bit_rotate_left(N, num, 3))
count_ones(N, num)

1011010
0101101
1110111
1010101


4

### Section 3: The simulation logic

In [2]:
class Pushups:
    def __init__(self, N, initial_state):
        # number of friends doing pushups
        self.N = N
        # bit representation of the initial state
        self.number = initial_state
        self.total_pushups = count_ones(N, initial_state)
        self.iterations = 1

        # track if we hit 0 pushups in an iteration
        self.terminated = False
        # track the two iterations where we repeat
        self.repeated = None

    def iterate(self):
        right = bit_rotate_right(self.N, self.number, 1)
        left = bit_rotate_left(self.N, self.number, 1)
        self.number = xor(self.N, right, left)
        self.total_pushups += count_ones(self.N, self.number)
        self.iterations += 1

    def print(self, as_blocks=True):
        print_bits(self.N, self.number, as_blocks=as_blocks)

    def print_iters(self, iters):
        for _ in range(iters):
            self.print()
            self.iterate()
        self.print()

    # TODO: add max_iters
    def iterate_until_repeat(self, print_output=False, as_blocks=True):
        seen = dict()
        while self.number not in seen:
            if print_output:
                self.print(as_blocks=as_blocks)
            seen[self.number] = self.iterations
            self.iterate()
        if print_output:
            self.print(as_blocks=as_blocks)
        
        self.repeated = (seen[self.number], self.iterations)
        self.terminated = (self.number == 0)
        self.summary()

    def summary(self):
        if self.repeated is not None:
            if self.terminated:
                print(f'(N={self.N}) Terminated at iteration {self.iterations - 1} with {self.total_pushups} pushups')
            else:
                print(f'(N={self.N}) Repeated at iteration {self.iterations} with {self.total_pushups} pushups')
                print(f'\tIterations {self.repeated[0]} and {self.repeated[1]} are the same')

        else:
            print(f'(N={self.N}) No termination or repeat after {self.iterations} iterations and {self.total_pushups} pushups')

pushups = Pushups(10, 0b1)
pushups.iterate_until_repeat(print_output=True, as_blocks=False)

0000000001
1000000010
0100000100
1010001010
0001010000
0010001000
0101010100
1000000010
(N=10) Repeated at iteration 8 with 19 pushups
	Iterations 2 and 8 are the same


### Section 4: Trying it out

In [3]:
for N in range(1, 33):
    pushups = Pushups(N, 0b1)
    pushups.iterate_until_repeat(print_output=False)

(N=1) Terminated at iteration 2 with 1 pushups
(N=2) Terminated at iteration 2 with 1 pushups
(N=3) Repeated at iteration 3 with 5 pushups
	Iterations 2 and 3 are the same
(N=4) Terminated at iteration 3 with 3 pushups
(N=5) Repeated at iteration 5 with 11 pushups
	Iterations 2 and 5 are the same
(N=6) Repeated at iteration 4 with 7 pushups
	Iterations 2 and 4 are the same
(N=7) Repeated at iteration 9 with 27 pushups
	Iterations 2 and 9 are the same
(N=8) Terminated at iteration 5 with 9 pushups
(N=9) Repeated at iteration 9 with 29 pushups
	Iterations 2 and 9 are the same
(N=10) Repeated at iteration 8 with 19 pushups
	Iterations 2 and 8 are the same
(N=11) Repeated at iteration 33 with 163 pushups
	Iterations 2 and 33 are the same
(N=12) Repeated at iteration 7 with 17 pushups
	Iterations 3 and 7 are the same
(N=13) Repeated at iteration 65 with 387 pushups
	Iterations 2 and 65 are the same
(N=14) Repeated at iteration 16 with 51 pushups
	Iterations 2 and 16 are the same
(N=15) Repe

### Section 5: Notice a pattern?
Here are the the cases that terminated: 1, 2, 4, 8, 16, 32

It may not be surprising that the powers of two all terminate, but it is interesting that they're the *only* ones that terminate.

## Part 2: Wait what?
### Section 1: WTF???
Let's visualize the output of the terminating case $N=64$.

In [4]:
pushups = Pushups(64, 1 << 32)
pushups.iterate_until_repeat(print_output=True, as_blocks=False)

0000000000000000000000000000000100000000000000000000000000000000
0000000000000000000000000000001010000000000000000000000000000000
0000000000000000000000000000010001000000000000000000000000000000
0000000000000000000000000000101010100000000000000000000000000000
0000000000000000000000000001000000010000000000000000000000000000
0000000000000000000000000010100000101000000000000000000000000000
0000000000000000000000000100010001000100000000000000000000000000
0000000000000000000000001010101010101010000000000000000000000000
0000000000000000000000010000000000000001000000000000000000000000
0000000000000000000000101000000000000010100000000000000000000000
0000000000000000000001000100000000000100010000000000000000000000
0000000000000000000010101010000000001010101000000000000000000000
0000000000000000000100000001000000010000000100000000000000000000
0000000000000000001010000010100000101000001010000000000000000000
0000000000000000010001000100010001000100010001000000000000000000
0000000000000000101010101

Do you see it? Let's make it a little more obvious. We'll turn the 1's into filled-in blocks and the 0's into empty space.

In [5]:
pushups = Pushups(64, 1 << 32)
pushups.iterate_until_repeat(print_output=True, as_blocks=True)

                               ■                                
                              ■ ■                               
                             ■   ■                              
                            ■ ■ ■ ■                             
                           ■       ■                            
                          ■ ■     ■ ■                           
                         ■   ■   ■   ■                          
                        ■ ■ ■ ■ ■ ■ ■ ■                         
                       ■               ■                        
                      ■ ■             ■ ■                       
                     ■   ■           ■   ■                      
                    ■ ■ ■ ■         ■ ■ ■ ■                     
                   ■       ■       ■       ■                    
                  ■ ■     ■ ■     ■ ■     ■ ■                   
                 ■   ■   ■   ■   ■   ■   ■   ■                  
                ■ ■ ■ ■ ■

### Section 2: The Sierpiński Triangle
Yes, you're seeing that right. That's the famous triangle. My mind was blown when I saw this.